### ベクトルストアでの利用
1. FAISSに格納
2. FAISSから取得
3. FAISSでベクトル検索

In [ ]:
import os
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# from langchain_community.vectorstores import Pinecone # ほかのベクトルストアもコード変更なく同様に使える
from pypdf import PdfReader
from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv('../.env')

In [ ]:
# 1. PDFデータ読み込み
pdf_page = PdfReader("./data/llm.pdf")
text = ""
# PDFデータをテキストに変換
for page in pdf_page.pages:
    text += page.extract_text()
print(text[:100]) # 先頭100文字を表示

In [ ]:
# 2. データをチャンクに小分けにする
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # チャンクの最大文字数
    chunk_overlap=100   # チャンク間の重複文字数
)
docs = text_splitter.split_text(text) # テキストをチャンクに分割
docs_chunks = text_splitter.create_documents(docs) # チャンクをドキュメントに変換
print("Docサイズ", len(docs_chunks))
print("型", type(docs_chunks[0]))

In [8]:
# 3. OpenAIのembeddingモデル取得
openAI_key = os.getenv('AZURE_OPENAI_API_KEY')
openAI_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

embeddings = None
if openAI_key != "" :
    embeddings = AzureOpenAIEmbeddings(
      api_key=openAI_key,
      azure_deployment="text-embedding-ada-002",
      # openai_api_versiton="2024-10-21",
      azure_endpoint=openAI_endpoint
    )
else:
    print("EmbeddingのAPIKeyを設定してください")

In [ ]:
# 4. ベクトル化
# 一括でベクトル化する場合
# faiss_db = FAISS.from_documents(documents=docs, embedding=embeddings)

# プログレスバーを表示しながらベクトル化
faiss_db = None
with tqdm(total=len(docs_chunks), desc="documents 抽出") as pbar:
    for d in docs_chunks:
        if faiss_db:
            faiss_db.add_documents([d])
        else:
            faiss_db = FAISS.from_documents([d], embeddings)
        pbar.update(1)


In [10]:
# 5. ベクトル化データを保存
faiss_db.save_local("./db")

In [11]:
# ベクトル化データを読み込み
vectorstore  = FAISS.load_local(
    "./db", # ベクトル化データの保存先
    embeddings, # ベクトル化モデル
    allow_dangerous_deserialization=True # デシリアライズを許可 (デフォルトはFalse)でセキュリティを向上
)

# retrieverを取得
retriever = vectorstore.as_retriever()

In [ ]:
# 類似文書の取得
similar_docs = retriever.get_relevant_documents("言語モデル", k=3)
# similar_docs = retriever.invoke("言語モデル", k=3)

# 類似文書を表示
for doc in similar_docs:
    print(doc)
    print("-----")